In [6]:
import csv                       #use library csv to open and read csv data
import numpy as np 
import math
import random
import datetime
np.set_printoptions(threshold=np.inf)

path='/home/mz/Desktop/breast-cancer-wisconsin.data'

with open(path,'r') as cancerdata:
    reader = csv.reader(cancerdata) 
    rows= [row for row in reader]
    
cdata=np.array(rows)               #change rows' datatype into array

todelete=np.where(cdata=='?')               #save row number of 16 arrays which need to be deleted
#print(todelete[0])
ddataset=np.delete(cdata, todelete[0], axis = 0) #delete those rows which contain ‘？’

#print("The type and shape of cdata is",type(cdata), cdata.shape)
#print("The type and shape of ddataset is",type(ddataset), ddataset.shape)
test=np.where(ddataset=='?')                #Check if there are still '?' in ndarray ddataset
#print("The new dataset is contains ? in ", test)
#print(ddataset)
#print('\n')
cdataset = ddataset.astype(np.float) #change datatype from string to float
#print(cdataset)

dataset = np.delete(cdataset,0,axis = 1) #delete the name
for i in range(dataset.shape[0]):
    if dataset[i,dataset.shape[1]-1]==2: dataset[i,dataset.shape[1]-1]=0
    else: dataset[i,dataset.shape[1]-1]=1
#print(dataset,dataset.shape)
#data0=dataset
datash = dataset
np.random.shuffle(datash)
data0=datash





k=5
n0 = data0.shape[0]
X_test = data0[0:int(n0/k), 0:-1 ]
y_test = data0[0:int(n0/k), -1 ]

data2 = np.delete(data0, range(0,int(n0/k)),axis = 0)#delete testset from data
X_all = data2[:,0:-1]
y_all = data2[:,-1]


n1 = data2.shape[0]#number of rows in data2 for training and verti
X_valid = [[]for i in range(k)]
X_train = [[]for i in range(k)]
y_valid = [[]for i in range(k)]
y_train = [[]for i in range(k)]
trainset = [[]for i in range(k)]
for i in range(0,k):#generate k training sets and validation sets
    X_valid[i] = np.array(data2[int((i*n1)/k):int((i+1)*n1/k), 0:-1])
    y_valid[i] = np.array(data2[int((i*n1)/k):int((i+1)*n1/k), -1])
    a=np.arange(int((i*n1)/k),int((i+1)*n1/k))
    trainset[i] = np.delete(data2, a , axis=0)
    X_train[i] = trainset[i][:,0:-1]
    y_train[i] = trainset[i][:,-1]
    print('valid i=', i, X_train[i].shape)
    print('valid i=', i, y_train[i].shape)

valid i= 0 (438, 9)
valid i= 0 (438,)
valid i= 1 (438, 9)
valid i= 1 (438,)
valid i= 2 (437, 9)
valid i= 2 (437,)
valid i= 3 (438, 9)
valid i= 3 (438,)
valid i= 4 (437, 9)
valid i= 4 (437,)


In [11]:
def sigmoid(gamma):#the sigmoid function we used in gradient descenting and prediction. To avoid overflowing, input of exp is always a negative number
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))
    

'''
def add(data,w):
            m = data.shape[1]
            summe = np.zeros(m-1)
            for i in range(0,data.shape[0]):
                a = data[i,0:(data.shape[1]-1)]*((data[i,(data.shape[1]-1)])-sigma(w, data[i,0:(data.shape[1]-1)]))
                summe = summe + a         
            return summe    
    
def gd(w,lr,data):
            neww = w + lr*add(data,w)
            return neww
'''
#lr learning rate; n iteration times; data data used for training; y_train:label of training data
def findw(lr,n,data,y_train):
    traindata = data
    m = traindata.shape[1]#column number of input data(features)
    #w = [0,0,0,0,0,0,0,0,0,0,0]#除去最后一列的quality
    #w = [1,1,1,1,1,1,1,1,1,1,1]
    w = np.ones(m)#define w
    #right = [0,0,0,0,0,0,0,0,0,0,0]
    #vector
    right = np.zeros(m)
    for i in range(1,n):#n times iteration
        
        for j in range(0,traindata.shape[0]):
            right = right + traindata[j,]*(y_train[j]-sigmoid(np.dot(w,traindata[j,])))
            
        for l in range(0,right.shape[0]):
            right[l]=right[l]*lr
        #print(right)
        w=w+right
        #print(w)
    return w


#data:testdata; w:w from findw function
def predict(data,w):#use w which calculated by findw function to predict the classification of input data
        n = data.shape[0]
        result = np.zeros(n)
        for i in range(0,n):
            prob = sigmoid(np.dot(w,data[i,]))
            #prob = (1/math.sqrt(2*pi*sigma(w,data[i,0:n-1]))) * math.exp(-(1-np.transpose(w)*data[i,0:n-1]/2(sigma(w,data[i,0:n-1])*sigma(w,data[i,0:n-1]))))
            if prob>=0.5: result[i]=1
        return result
#pred: result from predict function; y: real value(label of training data)            
def accuracy(pred, y):#pred is result from fct predict
    real = y
    
    if pred.shape != real.shape:
        print('Error!!!')
        
    acc=0
    for i in range(pred.shape[0]):
        if pred[i]==real[i]:
            acc=acc+1
    Accuracy=acc/pred.shape[0]   
    #print('Accuracy is ',Accuracy)
    return Accuracy

#result: result from predict function; y: real value
def conmat(result,y):#return confusion matrix for prediction
    rv = y
    print(result.shape)
    print(rv.shape)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(result.shape[0]):
        if ((result[i]==1)&(rv[i]==1)): TP = TP+1
        if ((result[i]==0)&(rv[i]==0)): TN = TN+1
        if ((result[i]==0)&(rv[i]==1)): FN = FN+1
        if ((result[i]==1)&(rv[i]==0)): FP = FP+1
    metrix = np.array([[TP,FP], [FN,TN]])
    return metrix
'''                    
def Accu(data):#the data is after prediction
        n = shape.data[0]
        m = shape.data[1]
        a = 0
        for i in range(0,n):
                if data[i,m-2]==data[i,m-1]: a=a+1
        Accuracy = a/n
        return Accuracy
  
    0
    
def kfold(k, data):
    n0 = data.shape[0]
    testset = data[0:int(n0/k), ]
    data2 = np.delete(data, range(0,int(n0/k)),axis = 0)#delete testset from data
    n1 = data2.shape[0]#number of rows in data2 for training and verti

    vertiset = [[]for i in range(k)]
    trainset = [[]for i in range(k)]
    
    for i in range(0,k):
        vertiset[i] = np.array(data2[int((i*n1)/k):int((i+1)*n1/k), :])
        print('verti i=', i, vertiset[i].shape)
        a=np.arange(int((i*n1)/k),int((i+1)*n1/k))
        trainset[i] = np.delete(data2, a , axis=0)
'''      
    
def fit():#using gird search to find a best pair of alpha and iterition n
    start = datetime.datetime.now()
    
    a_n = 3#group number of alpha
    iter_n = 3#group number of n
    alpha = np.zeros(9*a_n)
    for i in range(9):#vector of values of alpha

        alpha[i]= 1e-4*(i+1)
        alpha[i+9]= 1e-3*(i+1)
        alpha[i+18]= 1e-2*(i+1)
        #alpha[i+27]= 1e-1*(i+1)
        #alpha[i+72]= (i+1)*10
        #alpha[i+81]= (i+1)*100
    
    #n_iteration = [500, 1000, 1500]
    n_iteration = [1000, 1500, 2500]#vector of values of n
    
    pairs = [[]for i in range(a_n*iter_n*9)]#list to save all possible pairs
    n = 0
    
    #offer all possibilities for pairs of alpha and iteration
    for i in range(9*a_n):
        j = 0
        while j<iter_n:
            pairs[n] = [alpha[i], n_iteration[j]]
            n = n+1
            j = j+1
             
    print(pairs)  
    
    print('\n')
 
    '''
    #here we chose numwewant from 9*a_n*5
    para_pos = np.array(random.sample(range(0,a_n*iter_n*9), numwewant))#generate used pairs's position
    #result = np.zeros([80,256])
    '''
    
    
    A = np.zeros(9*a_n*iter_n)#the avg accuracy vector for all
    Accu = np.zeros([9*a_n*iter_n,5])#Accuracy for each validation
    result = [[]for i in range(9*a_n*iter_n)]#save the predict results
    for i in range(a_n*9*iter_n):
            para=pairs[i]
            print(para)
            for j in range(5):#5 fold validation
                lr = para[0]
                n = para[1]
                w_model = findw(lr,n,X_train[j],y_train[j])#find w according to X_train and y_train
                print('fold',j,'of model',i,'finished',w_model)
                result[i] = predict(X_valid[j],w_model)#using X_valid as input to produce prediction output
                Accu[i,j]= accuracy(result[i], y_valid[j])#compare result(output prediction) and real values y_valid
                print('Accuracy is',Accu[i,j])
                if j ==4: A[i] = np.sum(Accu[i,:])/5
                if j ==4: print('\033[1;35m Average accuracy is \033[0m',A[i])  #return the avg accuracy        
                print('\n')
            
            
  


        
    a = np.max(A)#find the highst accuracy
    pos_max = np.where(A==a)#find the location of best accuracy
    location = pos_max[0][0]#location is a number which used to locate which random number in random vector
    print('used position',location)
    #pairs[location] is the pair of hyperparameters we used
    bestalpha = pairs[location][0]
    bestn = pairs[location][1]
    print('used hyperpara for final training','alpha is',bestalpha,'iteration number is',bestn)
    
    #trainging with X_all(combination of X_train and X_valid to find W) and the parameters found before
    w_final = findw(bestalpha, bestn, X_all, y_all)
    final_result = predict(X_test,w_final)#using testset to verify and produce prediction results
    finalaccu = accuracy(final_result,y_test)#calculate the accuracy according to label y_test and prediction results
    finalco = conmat(final_result,y_test)
    print('The final accuracy is', finalaccu)
    print('The confusion matrix of prediction is\n', finalco)
    end = datetime.datetime.now()
    print ('used time is',end-start)
    return w_final    

In [7]:
fit()#first time

[[0.0001, 1000], [0.0001, 1500], [0.0001, 2500], [0.0002, 1000], [0.0002, 1500], [0.0002, 2500], [0.00030000000000000003, 1000], [0.00030000000000000003, 1500], [0.00030000000000000003, 2500], [0.0004, 1000], [0.0004, 1500], [0.0004, 2500], [0.0005, 1000], [0.0005, 1500], [0.0005, 2500], [0.0006000000000000001, 1000], [0.0006000000000000001, 1500], [0.0006000000000000001, 2500], [0.0007, 1000], [0.0007, 1500], [0.0007, 2500], [0.0008, 1000], [0.0008, 1500], [0.0008, 2500], [0.0009000000000000001, 1000], [0.0009000000000000001, 1500], [0.0009000000000000001, 2500], [0.001, 1000], [0.001, 1500], [0.001, 2500], [0.002, 1000], [0.002, 1500], [0.002, 2500], [0.003, 1000], [0.003, 1500], [0.003, 2500], [0.004, 1000], [0.004, 1500], [0.004, 2500], [0.005, 1000], [0.005, 1500], [0.005, 2500], [0.006, 1000], [0.006, 1500], [0.006, 2500], [0.007, 1000], [0.007, 1500], [0.007, 2500], [0.008, 1000], [0.008, 1500], [0.008, 2500], [0.009000000000000001, 1000], [0.009000000000000001, 1500], [0.009000

fold 2 of model 7 finished [-0.2944472   0.87976884  0.10577281  0.17491519 -0.85274672  0.46337366
 -0.48028676  0.4158548  -0.17609701]
Accuracy is 0.8727272727272727


fold 3 of model 7 finished [-0.3103117   0.82028786  0.20516781  0.0250346  -0.84893542  0.55032763
 -0.38229617  0.30454519 -0.12340062]
Accuracy is 0.8532110091743119


fold 4 of model 7 finished [-0.32833648  1.25266962  0.13688444  0.12039634 -0.955564    0.53263529
 -0.49188662  0.37831471 -0.39186169]
Accuracy is 0.8272727272727273
 Average accuracy is  0.8629357798165138


[0.00030000000000000003, 2500]
fold 0 of model 8 finished [-0.27323168  0.90071297  0.08034885  0.19495894 -0.77270576  0.4579988
 -0.4161797   0.32132317 -0.29640663]
Accuracy is 0.8899082568807339


fold 1 of model 8 finished [-0.3622329   1.00495972  0.24487557  0.18447003 -0.92256903  0.64035804
 -0.56207786  0.35065519 -0.40206167]
Accuracy is 0.8715596330275229


fold 2 of model 8 finished [-0.2944472   0.87976955  0.10577244  0.1749151

fold 2 of model 16 finished [-0.42677557  1.0640809   0.12088845  0.18090052 -1.14801673  0.55522947
 -0.62224082  0.51866256 -0.27203887]
Accuracy is 0.8636363636363636


fold 3 of model 16 finished [-0.43297925  0.96216302  0.25773111 -0.00199957 -1.13504785  0.67068008
 -0.51285229  0.37723805 -0.21064345]
Accuracy is 0.8440366972477065


fold 4 of model 16 finished [-0.39167092  1.34535065  0.14053933  0.10829707 -1.09425831  0.56271055
 -0.5580645   0.38911908 -0.45402363]
Accuracy is 0.8363636363636363
 Average accuracy is  0.8555963302752293


[0.0006000000000000001, 2500]
fold 0 of model 17 finished [-0.41144059  1.14405749  0.10146015  0.20645137 -1.15212181  0.598305
 -0.59527879  0.42173762 -0.41810434]
Accuracy is 0.8899082568807339


fold 1 of model 17 finished [-0.43325324  1.10039558  0.25661998  0.17373077 -1.0896974   0.6935768
 -0.64382635  0.37022786 -0.46913605]
Accuracy is 0.8440366972477065


fold 2 of model 17 finished [-0.42677557  1.0640809   0.12088845  0.1809

fold 2 of model 25 finished [-0.62173944  1.4581429   0.15774817  0.23852102 -1.62502413  0.77386734
 -0.85378009  0.7260104  -0.41773427]
Accuracy is 0.8454545454545455


fold 3 of model 25 finished [-0.62763516  1.29108375  0.37867633 -0.03651079 -1.627659    0.95469882
 -0.71448821  0.55257274 -0.37149732]
Accuracy is 0.8440366972477065


fold 4 of model 25 finished [-0.55846396  1.75596642  0.20537126  0.11913154 -1.56694741  0.76610338
 -0.73647226  0.50790355 -0.64482024]
Accuracy is 0.8272727272727273
 Average accuracy is  0.8519766472060051


[0.0009000000000000001, 2500]
fold 0 of model 26 finished [-0.60271496  1.57222003  0.15047797  0.26215458 -1.67234995  0.84943551
 -0.83391679  0.60240349 -0.55945663]
Accuracy is 0.8990825688073395


fold 1 of model 26 finished [-0.62941099  1.49281493  0.34083296  0.19478884 -1.6082137   0.96554008
 -0.87888991  0.52051316 -0.63961852]
Accuracy is 0.8440366972477065


fold 2 of model 26 finished [-0.62173944  1.4581429   0.15774817  0.2

fold 2 of model 34 finished [-1.72387805  4.42760084  0.64901812  0.71670667 -4.77470099  2.26896164
 -2.40639575  2.25271566 -0.90679772]
Accuracy is 0.8727272727272727


fold 3 of model 34 finished [-1.61834716  4.65756691  1.62487116  0.30300149 -4.62406624  3.38865975
 -1.81891297  2.08518456 -0.79431709]
Accuracy is 0.8532110091743119


fold 4 of model 34 finished [-2.21515904  6.08005639  1.10559237  0.65940458 -5.1044947   3.24797456
 -2.37392418  1.88624543 -2.26937091]
Accuracy is 0.8181818181818182
 Average accuracy is  0.8427689741451209


[0.003, 2500]
fold 0 of model 35 finished [-1.09137471  5.33312463  0.9181872   1.39797905 -4.43310742  2.90383235
 -2.00312864  2.22894639 -1.3931048 ]
Accuracy is 0.8440366972477065


fold 1 of model 35 finished [-2.34394064  5.25929631  1.45464443  0.76336712 -5.02527033  3.54006493
 -2.82907287  2.02662286 -2.03757461]
Accuracy is 0.8532110091743119


fold 2 of model 35 finished [-2.3527397   4.7293589   0.96703988  0.87301152 -4.78131

...

fold 0 of model 52 finished [ -6.16725853  13.83511904   0.75521936   2.33190473 -14.09543365
   6.65068332  -7.52595723   4.60431054  -5.11601575]
Accuracy is 0.8990825688073395


fold 1 of model 52 finished [ -3.67691486  14.82503914   4.09193356   2.95087131 -12.19131224
   9.15593907  -6.55916705   5.6558492   -5.02032028]
Accuracy is 0.8623853211009175


fold 2 of model 52 finished [ -4.89239284  13.40854989   2.2105918    2.44311871 -13.65173558
   6.84160889  -6.70077363   6.43988884  -2.69047196]
Accuracy is 0.8636363636363636


fold 3 of model 52 finished [ -5.16459127  13.38631941   4.68900162   1.02120665 -13.52750297
  10.45829712  -5.16222627   6.22854262  -2.65934937]
Accuracy is 0.8532110091743119


fold 4 of model 52 finished [ -2.82787405  17.41612629   2.89330912   1.88281356 -11.48206857
   7.2315253   -4.61099833   5.33421166  -4.37732037]
Accuracy is 0.7272727272727273
 Average accuracy is  0.841117597998332


[0.009000000000000001, 2500]
fold 0 of model 53 finishe

fold 3 of model 60 finished [-18.18630439  44.88201421  16.23136067   3.60835678 -44.33231967
  34.61186907 -17.48286027  20.92991393  -8.04549669]
Accuracy is 0.8532110091743119


fold 4 of model 60 finished [-19.93379978  54.4929504    7.75907104   4.36489938 -44.79031263
  25.91807082 -20.84806902  16.09137081 -19.04400747]
Accuracy is 0.8363636363636363
 Average accuracy is  0.818999165971643


[0.03, 1500]
fold 0 of model 61 finished [-15.38325736  49.55781704   6.84697045  11.98843515 -43.81989958
  25.78696779 -21.38271218  19.30911437 -15.58712548]
Accuracy is 0.8807339449541285


fold 1 of model 61 finished [-22.24293445  47.04925072  13.04284407   7.25741506 -44.23900135
  31.56276819 -25.98635563  18.68069643 -18.17737849]
Accuracy is 0.8440366972477065


fold 2 of model 61 finished [-14.36007311  40.84558676   5.98366164   6.67954699 -41.7369094
  19.76647252 -19.79054134  18.7920334   -7.81823744]
Accuracy is 0.8818181818181818


fold 3 of model 61 finished [-19.55432479  

fold 1 of model 69 finished [-44.48616698  96.53975123  29.3193557   16.63297623 -87.02027769
  66.67703934 -52.03309609  38.01118885 -36.26364173]
Accuracy is 0.8623853211009175


fold 2 of model 69 finished [-36.39031012  59.69759915  -6.70000035   0.85988878 -80.69926328
  18.95759292 -41.01455866  20.29028582 -18.88498611]
Accuracy is 0.6181818181818182


fold 3 of model 69 finished [-35.9498294   77.82254527  22.83091663   0.58744506 -87.88121428
  52.33671578 -36.5312524   31.17140679 -16.2921855 ]
Accuracy is 0.8440366972477065


fold 4 of model 69 finished [-20.57917177 109.11006154  18.13192231  10.39460617 -74.18031081
  45.38086428 -29.31468952  34.37141779 -29.21088242]
Accuracy is 0.7818181818181819
 Average accuracy is  0.7754128440366972


[0.06, 1500]
fold 0 of model 70 finished [-45.0232996   93.1465304    8.8897392   17.74616049 -96.83744537
  51.94360472 -49.24708532  36.05300576 -36.02210083]
Accuracy is 0.8990825688073395


fold 1 of model 70 finished [ -3.52742838

fold 3 of model 77 finished [ -43.68126954   89.60300167   19.12082458   -3.87613799 -113.34242572
   54.7139561   -47.63482648   33.44373159  -21.22547588]
Accuracy is 0.8165137614678899


fold 4 of model 77 finished [ -35.59053798  147.52592933   27.00329514   13.80700695 -104.44862174
   67.54427829  -42.06412295   47.0738669   -42.11037614]
Accuracy is 0.8181818181818182
 Average accuracy is  0.8134278565471227


[0.09, 1000]
fold 0 of model 78 finished [ -28.31637536  142.77204306   24.04439871   39.42530137 -113.57880279
   73.41005558  -52.95541849   58.1873753   -40.03186562]
Accuracy is 0.8348623853211009


fold 1 of model 78 finished [ -51.07449815  115.53962157   22.53232883   14.94109312 -120.23409132
   64.8847221   -73.40995357   37.14569605  -50.27798331]
Accuracy is 0.7981651376146789


fold 2 of model 78 finished [ -43.04619803  113.6316244    12.64046461   18.67427387 -122.8502807
   53.07033253  -59.23879466   57.68401465  -22.06051248]
Accuracy is 0.8636363636363636

array([ -31.69388304,  147.46983322,   33.31713968,   27.56135518,
       -129.05224249,   80.73920975,  -52.84585681,   59.71864271,
        -36.99392484])

In [10]:
fit()#second time

[[0.0001, 1000], [0.0001, 1500], [0.0001, 2500], [0.0002, 1000], [0.0002, 1500], [0.0002, 2500], [0.00030000000000000003, 1000], [0.00030000000000000003, 1500], [0.00030000000000000003, 2500], [0.0004, 1000], [0.0004, 1500], [0.0004, 2500], [0.0005, 1000], [0.0005, 1500], [0.0005, 2500], [0.0006000000000000001, 1000], [0.0006000000000000001, 1500], [0.0006000000000000001, 2500], [0.0007, 1000], [0.0007, 1500], [0.0007, 2500], [0.0008, 1000], [0.0008, 1500], [0.0008, 2500], [0.0009000000000000001, 1000], [0.0009000000000000001, 1500], [0.0009000000000000001, 2500], [0.001, 1000], [0.001, 1500], [0.001, 2500], [0.002, 1000], [0.002, 1500], [0.002, 2500], [0.003, 1000], [0.003, 1500], [0.003, 2500], [0.004, 1000], [0.004, 1500], [0.004, 2500], [0.005, 1000], [0.005, 1500], [0.005, 2500], [0.006, 1000], [0.006, 1500], [0.006, 2500], [0.007, 1000], [0.007, 1500], [0.007, 2500], [0.008, 1000], [0.008, 1500], [0.008, 2500], [0.009000000000000001, 1000], [0.009000000000000001, 1500], [0.009000

fold 2 of model 7 finished [-0.42908118  1.07339175  0.23071932  0.18433848 -0.91806336  0.6620991
 -0.63211098  0.35880153 -0.27091519]
Accuracy is 0.8363636363636363


fold 3 of model 7 finished [-0.36116     0.89911863  0.20129495  0.21133719 -0.84114174  0.51582113
 -0.47870722  0.3153608  -0.20368317]
Accuracy is 0.9174311926605505


fold 4 of model 7 finished [-0.3214272   1.12324483  0.07747614  0.15487044 -0.8222487   0.44649124
 -0.51629122  0.35835722 -0.42566229]
Accuracy is 0.8545454545454545
 Average accuracy is  0.8519432860717264


[0.00030000000000000003, 2500]
fold 0 of model 8 finished [-0.29981612  1.04695566  0.13074532  0.29149516 -0.89371617  0.49728889
 -0.61737106  0.33547641 -0.21261081]
Accuracy is 0.8440366972477065


fold 1 of model 8 finished [-0.36153407  0.85143238  0.29303447  0.26453638 -0.73257076  0.51697697
 -0.63243451  0.26666267 -0.23867094]
Accuracy is 0.8073394495412844


fold 2 of model 8 finished [-0.42908129  1.07340247  0.23071457  0.1843384

fold 1 of model 16 finished [-0.36610413  1.1013939   0.38835153  0.34028758 -0.8760183   0.67492221
 -0.69192282  0.35960362 -0.24927693]
Accuracy is 0.8165137614678899


fold 2 of model 16 finished [-0.49476235  1.14133129  0.24791437  0.17183236 -1.0353963   0.6916047
 -0.69334523  0.36347408 -0.31214324]
Accuracy is 0.8727272727272727


fold 3 of model 16 finished [-0.36644465  1.151396    0.32898464  0.30932656 -1.06466555  0.70126526
 -0.50401422  0.43768886 -0.22517087]
Accuracy is 0.8715596330275229


fold 4 of model 16 finished [-0.32132676  1.31484116  0.15352946  0.18886871 -0.91991146  0.54602415
 -0.52423844  0.42907582 -0.45989833]
Accuracy is 0.8727272727272727
 Average accuracy is  0.8555129274395329


[0.0006000000000000001, 2500]
fold 0 of model 17 finished [-0.39349405  1.17400725  0.14932162  0.3168848  -1.08735219  0.52748505
 -0.7121137   0.36838803 -0.25538926]
Accuracy is 0.8440366972477065


fold 1 of model 17 finished [-0.36610413  1.1013939   0.38835153  0.34

fold 0 of model 25 finished [-0.6096351   1.65657584  0.23341751  0.43653332 -1.63446432  0.68917061
 -0.98433492  0.539623   -0.33718161]
Accuracy is 0.8256880733944955


fold 1 of model 25 finished [-0.69399805  1.44017884  0.40965791  0.34926523 -1.4322277   0.83198911
 -1.06636129  0.39201472 -0.34497871]
Accuracy is 0.8440366972477065


fold 2 of model 25 finished [-0.71550378  1.52843003  0.38370642  0.18648467 -1.52036802  0.92461598
 -0.92228055  0.47865822 -0.4519811 ]
Accuracy is 0.8727272727272727


fold 3 of model 25 finished [-0.72443205  1.47467549  0.36291741  0.30260659 -1.72217843  0.87637778
 -0.82530267  0.51123086 -0.40616645]
Accuracy is 0.926605504587156


fold 4 of model 25 finished [-0.61383226  1.70906321  0.17296656  0.11606104 -1.42558281  0.66151616
 -0.78461545  0.48775082 -0.65396295]
Accuracy is 0.8090909090909091
 Average accuracy is  0.8556296914095078


[0.0009000000000000001, 2500]
fold 0 of model 26 finished [-0.6096351   1.65657584  0.23341751  0.43

fold 0 of model 34 finished [-2.5285767   5.45135604  1.20430248  1.44731506 -5.32753846  2.70338765
 -3.25549813  1.95661581 -1.43717784]
Accuracy is 0.8440366972477065


fold 1 of model 34 finished [-1.94069632  5.34261761  2.16473441  1.73846538 -3.730697    3.64886749
 -2.89944699  1.94914319 -1.1167669 ]
Accuracy is 0.7706422018348624


fold 2 of model 34 finished [-2.18533747  4.42083598  0.96437797  0.50228356 -4.43557936  2.45797548
 -2.66136302  1.30045801 -1.3223807 ]
Accuracy is 0.8545454545454545


fold 3 of model 34 finished [-2.3172983   5.48017824  1.76702757  1.42591388 -5.14531962  3.60875073
 -2.39687588  2.20666501 -1.3474163 ]
Accuracy is 0.8715596330275229


fold 4 of model 34 finished [-2.12866983  4.48012016  0.68295211  0.62704947 -3.94784326  1.99624644
 -2.21455088  1.52324074 -2.1509759 ]
Accuracy is 0.8090909090909091
 Average accuracy is  0.8299749791492911


[0.003, 2500]
fold 0 of model 35 finished [-1.72587118  5.12745889  0.95021843  1.54086472 -4.67222

...

fold 4 of model 51 finished [ 4.95489038 16.23328326  6.20918541  7.12272876 -2.07145253  9.67933785
  2.52191119  8.7253955  -1.54401931]
Accuracy is 0.3181818181818182
 Average accuracy is  0.7135279399499583


[0.009000000000000001, 1500]
fold 0 of model 52 finished [ -3.53951883  14.14840848   2.96642596   4.64194951 -11.61366585
   6.66821106  -6.38051403   4.69989834  -2.78564112]
Accuracy is 0.8256880733944955


fold 1 of model 52 finished [ -7.44576925  11.18315138   2.67277432   2.71508666 -11.95416063
   5.49173971 -10.39201691   2.61454964  -4.60632264]
Accuracy is 0.7798165137614679


fold 2 of model 52 finished [ -4.15795485  16.35156457   6.84003084   4.29680175 -10.37521627
  11.52081595  -5.1049306    7.02693735  -2.69306969]
Accuracy is 0.5181818181818182


fold 3 of model 52 finished [ -6.88728973  14.10872581   4.11805503   3.54749068 -15.03829774
   8.56592101  -7.04955743   5.46401719  -3.78675742]
Accuracy is 0.908256880733945


fold 4 of model 52 finished [ 1.801

fold 2 of model 60 finished [-24.68630966  46.86047076  14.58233164   7.30761146 -41.84300542
  28.73179576 -25.38010769  15.65108149 -13.06402558]
Accuracy is 0.8454545454545455


fold 3 of model 60 finished [-14.44799388  44.1139639   13.40313143  12.54328529 -39.48738188
  23.90990255 -16.71320346  16.97057098  -8.5496078 ]
Accuracy is 0.8532110091743119


fold 4 of model 60 finished [-18.81060007  47.05503099   9.27341793   8.66277981 -35.84145331
  21.62478115 -19.33744592  16.35452415 -19.42453099]
Accuracy is 0.8363636363636363
 Average accuracy is  0.8299416180150125


[0.03, 1500]
fold 0 of model 61 finished [-15.91992972  42.75963803   5.12906882  11.59254835 -40.16782576
  17.15578001 -24.33260432  12.41726496 -10.89131807]
Accuracy is 0.8165137614678899


fold 1 of model 61 finished [-22.05777223  47.45366422  19.07343355  16.14449458 -36.69148392
  31.4594476  -31.30004336  16.80185076 -14.06833075]
Accuracy is 0.8165137614678899


fold 2 of model 61 finished [-24.44496183

fold 0 of model 69 finished [-34.98069441  92.27275644  19.68005723  28.77392373 -83.57484624
  46.45177069 -49.60742673  32.86006344 -22.37259464]
Accuracy is 0.8532110091743119


fold 1 of model 69 finished [-55.02518705  89.63981828  32.85799438  26.69315547 -79.9625032
  62.01859776 -66.59131889  29.08580119 -30.02294569]
Accuracy is 0.8532110091743119


fold 2 of model 69 finished [-53.35040555  94.28034962  34.6416094   17.73776763 -81.38607893
  62.27897292 -50.98052792  33.964761   -29.24305136]
Accuracy is 0.8363636363636363


fold 3 of model 69 finished [-38.94845219  91.98187077  30.58751851  24.3829999  -83.86882709
  54.70103277 -39.47697321  37.98476883 -21.08167774]
Accuracy is 0.8623853211009175


fold 4 of model 69 finished [ 13.01212289  90.40839008  23.17693391  32.42662931 -31.03879084
  44.12518364  -2.41900903  39.3919704  -16.86341059]
Accuracy is 0.3181818181818182
 Average accuracy is  0.7446705587989992


[0.06, 1500]
fold 0 of model 70 finished [-40.55757173 

fold 3 of model 77 finished [ -47.88571445  119.15974677   38.58816104   33.47535836 -104.11589949
   67.68770824  -50.95582309   46.30748746  -26.32750803]
Accuracy is 0.8715596330275229


fold 4 of model 77 finished [-29.09581957 108.64832161  11.21323202  20.62741362 -83.72955282
  37.76997638 -41.21322242  37.19053766 -42.64242674]
Accuracy is 0.8545454545454545
 Average accuracy is  0.8482902418682234


[0.09, 1000]
fold 0 of model 78 finished [ -70.29572367  119.37476324    9.0605043    29.17518333 -136.05855512
   45.65383445  -87.66329088   31.70533891  -37.94106862]
Accuracy is 0.7431192660550459


fold 1 of model 78 finished [-35.29978522 124.59714365  41.95219403  39.49044865 -86.43246051
  63.41475634 -75.87472956  40.19437274 -30.31747262]
Accuracy is 0.8073394495412844


fold 2 of model 78 finished [ -66.82945989  148.00791844   50.49306427   22.96269323 -130.2438681
   97.44185902  -77.7939441    54.55528891  -40.64275065]
Accuracy is 0.8454545454545455


fold 3 of model

array([-2.6001293 ,  4.30967288,  1.04229412,  0.76979301, -4.2647521 ,
        2.41204646, -2.72600677,  1.37316286, -1.44834793])

In [13]:
fit()#third time

[[0.0001, 1000], [0.0001, 1500], [0.0001, 2500], [0.0002, 1000], [0.0002, 1500], [0.0002, 2500], [0.00030000000000000003, 1000], [0.00030000000000000003, 1500], [0.00030000000000000003, 2500], [0.0004, 1000], [0.0004, 1500], [0.0004, 2500], [0.0005, 1000], [0.0005, 1500], [0.0005, 2500], [0.0006000000000000001, 1000], [0.0006000000000000001, 1500], [0.0006000000000000001, 2500], [0.0007, 1000], [0.0007, 1500], [0.0007, 2500], [0.0008, 1000], [0.0008, 1500], [0.0008, 2500], [0.0009000000000000001, 1000], [0.0009000000000000001, 1500], [0.0009000000000000001, 2500], [0.001, 1000], [0.001, 1500], [0.001, 2500], [0.002, 1000], [0.002, 1500], [0.002, 2500], [0.003, 1000], [0.003, 1500], [0.003, 2500], [0.004, 1000], [0.004, 1500], [0.004, 2500], [0.005, 1000], [0.005, 1500], [0.005, 2500], [0.006, 1000], [0.006, 1500], [0.006, 2500], [0.007, 1000], [0.007, 1500], [0.007, 2500], [0.008, 1000], [0.008, 1500], [0.008, 2500], [0.009000000000000001, 1000], [0.009000000000000001, 1500], [0.009000

fold 2 of model 7 finished [-0.334817    1.02035671  0.10111317  0.04042753 -0.95459877  0.62445707
 -0.45580281  0.32333153 -0.00764529]
Accuracy is 0.8727272727272727


fold 3 of model 7 finished [-0.35985665  0.94122701  0.13457628  0.09993077 -0.85000614  0.59790217
 -0.50519889  0.36645556 -0.13673685]
Accuracy is 0.8256880733944955


fold 4 of model 7 finished [-0.34169059  1.02720191  0.0555735   0.12242942 -0.86767435  0.54808021
 -0.46087298  0.43672776 -0.21739633]
Accuracy is 0.8909090909090909
 Average accuracy is  0.868323603002502


[0.00030000000000000003, 2500]
fold 0 of model 8 finished [-0.37336564  0.90731496  0.18731828  0.19063477 -0.99644875  0.65167781
 -0.45871137  0.37490747 -0.1890208 ]
Accuracy is 0.8807339449541285


fold 1 of model 8 finished [-0.39084698  1.16305074  0.19511424  0.11821181 -1.05303401  0.68217077
 -0.550973    0.39654146 -0.28387001]
Accuracy is 0.8715596330275229


fold 2 of model 8 finished [-0.33481715  1.02036415  0.10110982  0.0404268

fold 1 of model 16 finished [-0.4452014   1.22352382  0.20791705  0.09933148 -1.17475945  0.7072292
 -0.60789431  0.41269934 -0.31497226]
Accuracy is 0.8623853211009175


fold 2 of model 16 finished [-0.46308488  1.18812725  0.14377417  0.0391184  -1.26096623  0.7191343
 -0.5817434   0.38994796 -0.02748196]
Accuracy is 0.8727272727272727


fold 3 of model 16 finished [-0.45546521  1.04305801  0.15482447  0.08287969 -1.05817124  0.67901499
 -0.61452989  0.40948159 -0.15716071]
Accuracy is 0.8165137614678899


fold 4 of model 16 finished [-0.44662101  1.17241702  0.02519806  0.11931794 -1.07709867  0.60986962
 -0.55772912  0.52306132 -0.26351661]
Accuracy is 0.8727272727272727
 Average accuracy is  0.857347789824854


[0.0006000000000000001, 2500]
fold 0 of model 17 finished [-0.44055627  0.9677674   0.19941003  0.18401087 -1.15741264  0.69971146
 -0.52896495  0.40713094 -0.22591218]
Accuracy is 0.8623853211009175


fold 1 of model 17 finished [-0.4452014   1.22352382  0.20791705  0.0993

fold 0 of model 25 finished [-0.63904749  1.27608394  0.29445233  0.21279696 -1.6896387   0.97310291
 -0.7288499   0.59745599 -0.36631322]
Accuracy is 0.8715596330275229


fold 1 of model 25 finished [-0.6281966   1.59879755  0.3349649   0.08655814 -1.72538322  0.93210739
 -0.82256477  0.59927137 -0.46542319]
Accuracy is 0.8532110091743119


fold 2 of model 25 finished [-0.67462162  1.62794903  0.20958884  0.0658212  -1.79697287  0.96838762
 -0.8115062   0.55380976 -0.06649067]
Accuracy is 0.8727272727272727


fold 3 of model 25 finished [-0.44111906  1.53355796  0.37965487  0.2356923  -1.33942759  1.10327883
 -0.66798191  0.68838226 -0.11830214]
Accuracy is 0.7339449541284404


fold 4 of model 25 finished [-0.42170624  1.79169239  0.13531945  0.31252603 -1.36070353  0.96473855
 -0.59094189  0.88074424 -0.27638455]
Accuracy is 0.7454545454545455
 Average accuracy is  0.8153794829024186


[0.0009000000000000001, 2500]
fold 0 of model 26 finished [-0.63904749  1.27608394  0.29445233  0.2

fold 0 of model 34 finished [-1.94582683  4.02010399  0.93317478  0.83021927 -5.02539578  2.87985693
 -2.14818717  1.75721096 -0.86851398]
Accuracy is 0.8623853211009175


fold 1 of model 34 finished [-2.21777557  5.39166019  1.37244805  0.34185715 -5.601407    3.33108827
 -2.61441874  2.26888529 -1.45534609]
Accuracy is 0.8623853211009175


fold 2 of model 34 finished [-2.63750521  5.88171899  0.92275453  0.36501307 -6.02356534  3.76980536
 -2.59356509  2.08541981 -0.18981224]
Accuracy is 0.8818181818181818


fold 3 of model 34 finished [-1.00787303  4.70283592  1.16247237  0.77858415 -3.76737709  3.09155421
 -1.93507335  2.09076927 -0.25745581]
Accuracy is 0.6880733944954128


fold 4 of model 34 finished [-0.95546486  5.46179189  0.38910086  0.94200522 -3.93691836  2.78812674
 -1.58505601  2.68872889 -0.72178689]
Accuracy is 0.6909090909090909
 Average accuracy is  0.7971142618849042


[0.003, 2500]
fold 0 of model 35 finished [-2.90505058  4.38574301  1.23780607  0.83779923 -5.52577

...

fold 4 of model 51 finished [ -3.39291481  15.44035878   1.53691389   2.6492666  -11.62975083
   8.01278648  -4.68822636   7.71448943  -2.33033041]
Accuracy is 0.7272727272727273
 Average accuracy is  0.7568974145120935


[0.009000000000000001, 1500]
fold 0 of model 52 finished [ -5.54832275  12.81227843   4.03695541   3.20672354 -14.57191239
  10.16661205  -6.07308908   5.48582274  -2.18743207]
Accuracy is 0.8623853211009175


fold 1 of model 52 finished [ -2.24907896  16.02232989   4.48968267   2.42783955 -12.46519866
   9.76340523  -4.62769865   7.0246381   -2.46240717]
Accuracy is 0.6788990825688074


fold 2 of model 52 finished [ -4.88300304  17.37008568   3.23910827   1.77158604 -15.73241743
  10.76671284  -6.38631953   6.17934218   0.93028495]
Accuracy is 0.8363636363636363


fold 3 of model 52 finished [ -6.79025111  14.8424581    4.54134252   2.35047104 -13.14331484
  11.35202172  -7.10940062   7.0683432   -2.03305898]
Accuracy is 0.8165137614678899


fold 4 of model 52 finish

fold 1 of model 60 finished [ -4.22168336  57.01316947  19.20415145  10.99763212 -38.40354812
  37.44090617 -11.63708831  27.54181485  -6.14992701]
Accuracy is 0.43119266055045874


fold 2 of model 60 finished [-17.79158078  50.77972823   9.05256756   2.6118637  -49.88986018
  31.87373749 -21.41024157  17.78997013   2.38501783]
Accuracy is 0.8636363636363636


fold 3 of model 60 finished [-20.76742519  48.19057036  13.92143002   7.83255656 -43.05589292
  35.93545293 -23.12410368  21.73993334  -7.12090985]
Accuracy is 0.8165137614678899


fold 4 of model 60 finished [-18.02500366  48.38369152   0.7573403    5.3737649  -44.16634524
  23.62667661 -19.45009523  23.8122623  -11.00210107]
Accuracy is 0.8909090909090909
 Average accuracy is  0.7729274395329441


[0.03, 1500]
fold 0 of model 61 finished [-26.1778235   33.14540441   4.58385584   3.69767054 -52.67924495
  23.63592163 -25.99422951  12.85652131 -11.37446175]
Accuracy is 0.6880733944954128


fold 1 of model 61 finished [-17.1611012

fold 4 of model 68 finished [-26.27728735  75.23707404  -1.78207105   7.79721777 -67.39423838
  32.63611999 -29.62916671  35.12867563 -16.29591531]
Accuracy is 0.8636363636363636
 Average accuracy is  0.723653044203503


[0.06, 1000]
fold 0 of model 69 finished [-43.46629341  84.04594996  27.24984227  20.28522726 -95.66046899
  67.76743105 -40.19023046  40.05677877 -19.16962839]
Accuracy is 0.8715596330275229


fold 1 of model 69 finished [ -46.63514982  109.89757229   30.89002067   13.57207534 -100.75397476
   69.82302366  -48.80869415   44.98607361  -24.87888595]
Accuracy is 0.8807339449541285


fold 2 of model 69 finished [-28.42430049 106.28953375  20.68078796  11.21340938 -96.24111428
  66.91549946 -39.11875494  37.2072043    8.917857  ]
Accuracy is 0.8363636363636363


fold 3 of model 69 finished [-41.13029199 103.34632527  34.19684924  21.46301267 -81.22704888
  82.82413094 -40.09595909  49.85270346 -11.66842723]
Accuracy is 0.7064220183486238


fold 4 of model 69 finished [-17.

fold 1 of model 77 finished [ -81.93978598  138.94443751   34.83693835   10.25346359 -145.00168435
   87.86211422  -75.24831662   58.1189173   -40.82379129]
Accuracy is 0.8532110091743119


fold 2 of model 77 finished [ -59.04956875  141.75692272   28.01780055   11.01695295 -136.62768391
   94.95537987  -61.21303033   51.34700776    4.38290304]
Accuracy is 0.8636363636363636


fold 3 of model 77 finished [-27.93932493 110.86420055  26.44769004  15.84145247 -97.54914111
  71.07610593 -51.50203924  45.31207327  -9.77305669]
Accuracy is 0.8073394495412844


fold 4 of model 77 finished [ -41.75112158  122.84682753   -2.53348689   14.26752817 -111.73138316
   54.46350806  -46.80360282   57.82754519  -27.22147814]
Accuracy is 0.8818181818181818
 Average accuracy is  0.8426688907422853


[0.09, 1000]
fold 0 of model 78 finished [ -46.33827971  112.81222022   32.03214839   28.63427744 -134.5884437
   90.1484383   -57.24377156   46.98312373  -20.0494168 ]
Accuracy is 0.8623853211009175


fold 1

array([-0.46959721,  1.14309818,  0.15481592,  0.10643147, -1.19700952,
        0.69736787, -0.59761912,  0.44933254, -0.20950594])

In [16]:
fit()#4th time

[[0.0001, 1000], [0.0001, 1500], [0.0001, 2500], [0.0002, 1000], [0.0002, 1500], [0.0002, 2500], [0.00030000000000000003, 1000], [0.00030000000000000003, 1500], [0.00030000000000000003, 2500], [0.0004, 1000], [0.0004, 1500], [0.0004, 2500], [0.0005, 1000], [0.0005, 1500], [0.0005, 2500], [0.0006000000000000001, 1000], [0.0006000000000000001, 1500], [0.0006000000000000001, 2500], [0.0007, 1000], [0.0007, 1500], [0.0007, 2500], [0.0008, 1000], [0.0008, 1500], [0.0008, 2500], [0.0009000000000000001, 1000], [0.0009000000000000001, 1500], [0.0009000000000000001, 2500], [0.001, 1000], [0.001, 1500], [0.001, 2500], [0.002, 1000], [0.002, 1500], [0.002, 2500], [0.003, 1000], [0.003, 1500], [0.003, 2500], [0.004, 1000], [0.004, 1500], [0.004, 2500], [0.005, 1000], [0.005, 1500], [0.005, 2500], [0.006, 1000], [0.006, 1500], [0.006, 2500], [0.007, 1000], [0.007, 1500], [0.007, 2500], [0.008, 1000], [0.008, 1500], [0.008, 2500], [0.009000000000000001, 1000], [0.009000000000000001, 1500], [0.009000

fold 3 of model 7 finished [-0.29524899  0.81728453  0.0922421   0.15922594 -0.78061203  0.50428331
 -0.45919961  0.398893   -0.27105397]
Accuracy is 0.9174311926605505


fold 4 of model 7 finished [-0.28942798  0.9612977   0.02572736  0.11882671 -0.80879374  0.57292106
 -0.64154422  0.45653374 -0.19334373]
Accuracy is 0.8727272727272727
 Average accuracy is  0.8756463719766472


[0.00030000000000000003, 2500]
fold 0 of model 8 finished [-0.3637512   1.08764594 -0.13373967  0.29080273 -0.84933753  0.5439462
 -0.61104988  0.54436035 -0.24943805]
Accuracy is 0.7889908256880734


fold 1 of model 8 finished [-0.3495642   0.94296017  0.10528026  0.17482213 -0.85142269  0.58381281
 -0.5123702   0.40995764 -0.3750095 ]
Accuracy is 0.8990825688073395


fold 2 of model 8 finished [-0.2668096   0.87499628  0.14103119  0.15853096 -0.83915366  0.57696262
 -0.58618123  0.49818265 -0.52141371]
Accuracy is 0.9


fold 3 of model 8 finished [-0.29524897  0.81728488  0.09224188  0.15922593 -0.7806122   

fold 3 of model 16 finished [-0.39723695  0.95976038  0.11612963  0.14005499 -0.99372755  0.5643824
 -0.56328897  0.45790382 -0.34647035]
Accuracy is 0.944954128440367


fold 4 of model 16 finished [-0.36169988  1.04244103  0.01785726  0.10973545 -0.95431307  0.60997597
 -0.70778973  0.49311353 -0.22835808]
Accuracy is 0.8636363636363636
 Average accuracy is  0.8757130942452044


[0.0006000000000000001, 2500]
fold 0 of model 17 finished [-0.35454713  1.16296216 -0.11512843  0.31277541 -0.87420763  0.58534066
 -0.61354477  0.58738239 -0.24432535]
Accuracy is 0.8073394495412844


fold 1 of model 17 finished [-0.33799759  1.02702635  0.14474818  0.19855844 -0.88861494  0.6393578
 -0.51503535  0.45606183 -0.38729718]
Accuracy is 0.8899082568807339


fold 2 of model 17 finished [-0.25809881  1.0085074   0.20124225  0.18885734 -0.89674943  0.65891954
 -0.60238413  0.57316572 -0.55800406]
Accuracy is 0.8727272727272727


fold 3 of model 17 finished [-0.39723695  0.95976038  0.11612963  0.1400

fold 3 of model 25 finished [-0.58917236  1.34855203  0.17965673  0.15304411 -1.47950899  0.78130076
 -0.76794251  0.65263473 -0.49936211]
Accuracy is 0.9541284403669725


fold 4 of model 25 finished [-0.53130785  1.38608246 -0.00676776  0.10203584 -1.39726052  0.78799394
 -0.9148289   0.67082999 -0.32383369]
Accuracy is 0.8454545454545455
 Average accuracy is  0.8592660550458715


[0.0009000000000000001, 2500]
fold 0 of model 26 finished [-0.42563597  1.66885499 -0.09734567  0.42911483 -1.15185489  0.83592626
 -0.73484056  0.86369191 -0.2924157 ]
Accuracy is 0.7889908256880734


fold 1 of model 26 finished [-0.54968695  1.35437589  0.16447999  0.18026162 -1.35394416  0.78424444
 -0.76141061  0.59153874 -0.57873653]
Accuracy is 0.8440366972477065


fold 2 of model 26 finished [-0.31417286  1.43142184  0.38633214  0.26273597 -1.22490664  0.95190636
 -0.7389193   0.8292962  -0.65728428]
Accuracy is 0.8545454545454545


fold 3 of model 26 finished [-0.58917236  1.34855203  0.17965673  0.1

fold 3 of model 34 finished [-2.33542081  4.57954734  1.2665413   0.79351496 -4.62028035  3.26316147
 -2.52659297  2.61439747 -1.85533252]
Accuracy is 0.8990825688073395


fold 4 of model 34 finished [-1.98560537  4.75242074  0.71751268  0.66620952 -4.1487383   3.18836926
 -2.93565436  2.54763145 -0.98658055]
Accuracy is 0.8818181818181818
 Average accuracy is  0.7665554628857382


[0.003, 2500]
fold 0 of model 35 finished [-2.8071299   3.98556273 -0.55466804  0.76824821 -4.10773717  2.01812392
 -2.62764338  1.95676565 -0.99321324]
Accuracy is 0.6972477064220184


fold 1 of model 35 finished [-2.54054093  4.88073697  1.16232247  0.75994115 -4.47347578  3.26748251
 -2.54648492  2.34572158 -1.95271855]
Accuracy is 0.8990825688073395


fold 2 of model 35 finished [-1.08373347  5.08328417  1.71905504  1.20432435 -3.48636031  3.65651784
 -2.02147785  3.07399135 -2.15881563]
Accuracy is 0.6545454545454545


fold 3 of model 35 finished [-0.99084507  3.47411953  0.61479077  0.72894457 -3.06539

...

fold 2 of model 52 finished [ -4.42073872  12.55353679   2.69821789   2.41705167 -11.65294222
   7.74293515  -7.56446731   7.08791341  -7.37550987]
Accuracy is 0.9


fold 3 of model 52 finished [ -4.56455083  11.66336802   2.45116717   2.53250959 -10.36557316
   6.68650432  -5.65953561   5.06010437  -4.04904572]
Accuracy is 0.908256880733945


fold 4 of model 52 finished [-2.9298024  11.27770359  0.65314837  1.641523   -9.63160265  6.19371677
 -7.02046994  5.48328468 -1.61456158]
Accuracy is 0.8727272727272727
 Average accuracy is  0.8572977481234363


[0.009000000000000001, 2500]
fold 0 of model 53 finished [ -7.84860948  14.99446175   1.44480423   3.91408565 -12.90456132
   9.39675096  -7.59553475   8.75631681  -2.77439636]
Accuracy is 0.8165137614678899


fold 1 of model 53 finished [ -4.79938508  12.26334014   0.99092423   1.75726777 -11.37100674
   6.06132965  -6.6414209    4.29902929  -4.52997316]
Accuracy is 0.8256880733944955


fold 2 of model 53 finished [ -5.91213703  10.2299

fold 0 of model 61 finished [ -7.79095932  40.85379591  -4.15332739  10.02936951 -31.40634058
  18.67510373 -14.95981631  20.56347292  -1.84705511]
Accuracy is 0.7798165137614679


fold 1 of model 61 finished [-15.66996656  40.17235129   3.14768312   5.29234859 -37.93950463
  19.97955209 -20.9752717   15.39941471 -15.38759929]
Accuracy is 0.8256880733944955


fold 2 of model 61 finished [ -7.03504597  38.2884659    8.20655849   7.4028822  -32.51410086
  23.00128444 -19.56629053  22.30106139 -18.30237685]
Accuracy is 0.8636363636363636


fold 3 of model 61 finished [-26.10193235  37.71125264   7.2182532    6.1743942  -42.90762007
  25.7363511  -24.1074991   17.28718367 -18.15013311]
Accuracy is 0.908256880733945


fold 4 of model 61 finished [ -9.08974389  35.33285412   1.83565206   4.95631754 -28.24204852
  19.32485404 -23.64289976  18.31756883  -4.07759584]
Accuracy is 0.8636363636363636
 Average accuracy is  0.848206839032527


[0.03, 2500]
fold 0 of model 62 finished [-17.67809292  

fold 3 of model 69 finished [-49.86979704  87.94694811  28.7585823   19.61563484 -80.28555985
  68.4874094  -43.13421456  46.08789508 -33.55315322]
Accuracy is 0.8899082568807339


fold 4 of model 69 finished [-39.69039868  85.30185858  17.15405474  14.24136829 -74.99281542
  62.48928832 -57.59873674  46.71451237 -17.45632732]
Accuracy is 0.8818181818181818
 Average accuracy is  0.818765638031693


[0.06, 1500]
fold 0 of model 70 finished [-25.76024703  78.13271942 -14.2096061   16.99691767 -69.13360249
  32.4231463  -36.80019011  36.46806573  -6.59233338]
Accuracy is 0.7614678899082569


fold 1 of model 70 finished [-31.95784582  74.8848206    4.36533638   7.94363747 -75.40535744
  36.28230218 -41.77063099  28.19292481 -31.09874039]
Accuracy is 0.8165137614678899


fold 2 of model 70 finished [-19.63233586  75.24103986  16.82767621  13.98110467 -67.15799305
  45.38993798 -40.44515234  44.08564495 -39.05772295]
Accuracy is 0.8636363636363636


fold 3 of model 70 finished [-25.58922624 

fold 0 of model 78 finished [ -90.98126428  138.50528973   -6.03510236   34.82072915 -135.88013771
   82.74306562  -79.7462581    75.14200431  -31.26826248]
Accuracy is 0.7706422018348624


fold 1 of model 78 finished [ -64.72894544  138.10782537   31.73942972   28.52446197 -120.40536481
   88.53394987  -68.87155122   59.44706408  -51.11730546]
Accuracy is 0.8899082568807339


fold 2 of model 78 finished [-17.20728798 110.57420198  20.34288933  26.29186685 -95.73767292
  64.39758889 -58.89252683  63.04104325 -60.17338151]
Accuracy is 0.8545454545454545


fold 3 of model 78 finished [-34.3992829  105.3190476   15.45074042  17.73225464 -94.29754199
  53.5055243  -49.17372653  43.33734387 -33.28524115]
Accuracy is 0.8990825688073395


fold 4 of model 78 finished [ -68.27117977  137.22274793   35.00370521   22.39940696 -114.41245539
  111.3007182   -84.47389022   82.22322826  -34.11901892]
Accuracy is 0.8636363636363636
 Average accuracy is  0.8555629691409508


[0.09, 1500]
fold 0 of mode

array([-12.62403119,  34.18727355,   4.70154203,   7.03811609,
       -29.40169643,  20.64761189, -17.3868891 ,  16.89541672,
        -9.44377994])

In [19]:
fit()#5th time

[[0.0001, 1000], [0.0001, 1500], [0.0001, 2500], [0.0002, 1000], [0.0002, 1500], [0.0002, 2500], [0.00030000000000000003, 1000], [0.00030000000000000003, 1500], [0.00030000000000000003, 2500], [0.0004, 1000], [0.0004, 1500], [0.0004, 2500], [0.0005, 1000], [0.0005, 1500], [0.0005, 2500], [0.0006000000000000001, 1000], [0.0006000000000000001, 1500], [0.0006000000000000001, 2500], [0.0007, 1000], [0.0007, 1500], [0.0007, 2500], [0.0008, 1000], [0.0008, 1500], [0.0008, 2500], [0.0009000000000000001, 1000], [0.0009000000000000001, 1500], [0.0009000000000000001, 2500], [0.001, 1000], [0.001, 1500], [0.001, 2500], [0.002, 1000], [0.002, 1500], [0.002, 2500], [0.003, 1000], [0.003, 1500], [0.003, 2500], [0.004, 1000], [0.004, 1500], [0.004, 2500], [0.005, 1000], [0.005, 1500], [0.005, 2500], [0.006, 1000], [0.006, 1500], [0.006, 2500], [0.007, 1000], [0.007, 1500], [0.007, 2500], [0.008, 1000], [0.008, 1500], [0.008, 2500], [0.009000000000000001, 1000], [0.009000000000000001, 1500], [0.009000

fold 2 of model 7 finished [-0.38281824  1.03854665  0.02492086 -0.06194246 -0.73123883  0.67156228
 -0.55324627  0.3918874  -0.18578272]
Accuracy is 0.8454545454545455


fold 3 of model 7 finished [-0.38322594  1.07120945  0.06007756  0.11472427 -0.83430496  0.61827807
 -0.52934057  0.39088812 -0.26195433]
Accuracy is 0.8807339449541285


fold 4 of model 7 finished [-0.39502147  0.85018049  0.23363177  0.11471436 -0.80148274  0.58354176
 -0.41556327  0.31728781 -0.31169409]
Accuracy is 0.8818181818181818
 Average accuracy is  0.8738949124270226


[0.00030000000000000003, 2500]
fold 0 of model 8 finished [-0.3450718   1.16550806  0.10326024  0.06700041 -0.93692961  0.63789992
 -0.59945312  0.41462092 -0.20923475]
Accuracy is 0.8623853211009175


fold 1 of model 8 finished [-0.4081783   0.81858141  0.17984415  0.03344495 -0.778207    0.60953039
 -0.45541701  0.40698591 -0.18110643]
Accuracy is 0.8990825688073395


fold 2 of model 8 finished [-0.38281875  1.03856431  0.02491086 -0.061945

fold 1 of model 16 finished [-0.53320215  0.97719845  0.2211529  -0.00301557 -1.02402829  0.71411819
 -0.5898214   0.47047259 -0.25516681]
Accuracy is 0.8990825688073395


fold 2 of model 16 finished [-0.45116912  1.12986653  0.02456569 -0.10071248 -0.87984903  0.72522447
 -0.62715319  0.41896517 -0.23872154]
Accuracy is 0.8272727272727273


fold 3 of model 16 finished [-0.44163409  1.14879215  0.06198115  0.09868    -0.95805316  0.64847194
 -0.60262484  0.41455882 -0.30448568]
Accuracy is 0.8899082568807339


fold 4 of model 16 finished [-0.39785943  1.08831296  0.33211848  0.16524353 -0.94498895  0.73405435
 -0.42399385  0.4108437  -0.35118886]
Accuracy is 0.8636363636363636
 Average accuracy is  0.8702919099249374


[0.0006000000000000001, 2500]
fold 0 of model 17 finished [-0.33307368  1.27681764  0.16370533  0.0982391  -1.00833944  0.71675039
 -0.61249931  0.47883179 -0.20528458]
Accuracy is 0.8715596330275229


fold 1 of model 17 finished [-0.53320215  0.97719845  0.2211529  -0.0

fold 0 of model 25 finished [-0.39652963  1.71824466  0.31955164  0.18883985 -1.34823373  1.00531616
 -0.73175762  0.68946472 -0.21403484]
Accuracy is 0.8348623853211009


fold 1 of model 25 finished [-0.73410706  1.33041468  0.31360419 -0.04188173 -1.45243005  0.96502694
 -0.78618097  0.62391869 -0.36807978]
Accuracy is 0.8899082568807339


fold 2 of model 25 finished [-0.41733251  1.63137202  0.20930928 -0.06085491 -1.11083725  1.10858743
 -0.64857184  0.67961048 -0.29751442]
Accuracy is 0.8272727272727273


fold 3 of model 25 finished [-0.42145811  1.63852707  0.24728451  0.22855344 -1.22504368  0.98690885
 -0.65376386  0.67218731 -0.33209746]
Accuracy is 0.8165137614678899


fold 4 of model 25 finished [-0.48725183  1.58027878  0.49763346  0.25586598 -1.30311098  1.05399636
 -0.49176369  0.59357866 -0.45177527]
Accuracy is 0.8363636363636363
 Average accuracy is  0.8409841534612175


[0.0009000000000000001, 2500]
fold 0 of model 26 finished [-0.39652963  1.71824466  0.31955164  0.1

fold 0 of model 34 finished [-2.05070663  5.09848592  0.83036431  0.31225898 -4.55902503  3.05694586
 -2.62059484  1.88875539 -0.8120833 ]
Accuracy is 0.8715596330275229


fold 1 of model 34 finished [-2.41022433  4.16585968  1.1261774   0.01196117 -4.13024553  3.17388362
 -2.36828243  2.04996797 -1.12793498]
Accuracy is 0.8899082568807339


fold 2 of model 34 finished [-2.03796378  5.04596987  0.47842453 -0.04753089 -3.53751186  3.43202578
 -2.34318535  1.99913022 -0.83645789]
Accuracy is 0.8636363636363636


fold 3 of model 34 finished [-1.83305828  4.89988536  0.68717474  0.56189372 -3.67328062  2.94327786
 -1.93246908  1.91564327 -1.15135951]
Accuracy is 0.8899082568807339


fold 4 of model 34 finished [-2.41293925  4.62131174  1.42094541  0.54148309 -4.45597857  3.0172214
 -1.8727147   1.67934633 -1.66898691]
Accuracy is 0.8818181818181818
 Average accuracy is  0.8793661384487074


[0.003, 2500]
fold 0 of model 35 finished [-2.26526845  5.47494767  1.14086279  0.47264834 -4.528043

...

fold 4 of model 51 finished [ -6.56527564  11.25425754   2.67683302   1.18203789 -11.83241327
   6.95366879  -4.86767064   3.75385203  -4.50680644]
Accuracy is 0.9
 Average accuracy is  0.883002502085071


[0.009000000000000001, 1500]
fold 0 of model 52 finished [ -2.97650179  13.73382152   2.47440063   1.30339203 -10.52335835
   7.82354473  -6.17542291   5.40938735  -1.65905683]
Accuracy is 0.8348623853211009


fold 1 of model 52 finished [ -7.30414193  10.38566175   1.68990376  -0.91353356 -12.5054559
   7.47661806  -7.43814102   4.45011384  -3.66333267]
Accuracy is 0.7798165137614679


fold 2 of model 52 finished [-6.1816295  16.0389822   3.02039219  1.24684091 -9.44758876 12.6955683
 -6.0246969   7.26532564 -2.66456039]
Accuracy is 0.7


fold 3 of model 52 finished [ -5.35527635  13.31392823   1.15529617   1.17713446 -10.5680232
   7.31674412  -5.77010684   4.84946511  -3.5772714 ]
Accuracy is 0.8899082568807339


fold 4 of model 52 finished [-3.22475257 16.68702663  8.31813772  4.

fold 2 of model 60 finished [-20.39349949  47.2439635    3.90563531  -0.56299581 -34.18712374
  33.70192004 -23.30839674  18.56281076  -9.60810985]
Accuracy is 0.8727272727272727


fold 3 of model 60 finished [-14.57013865  42.91622535   5.44504848   5.77530192 -32.61202892
  25.11133065 -16.34178668  17.08367631 -10.21991186]
Accuracy is 0.8623853211009175


fold 4 of model 60 finished [-24.80311318  44.00661947  15.39746355   4.96722963 -43.44625882
  31.33098486 -17.90159728  17.18734599 -17.50627565]
Accuracy is 0.8909090909090909
 Average accuracy is  0.8756630525437865


[0.03, 1500]
fold 0 of model 61 finished [-16.25984815  45.99978446   7.73586436   2.92369151 -40.37473622
  28.63160572 -24.33274069  17.55993985  -7.54170612]
Accuracy is 0.8623853211009175


fold 1 of model 61 finished [-25.49167148  39.41954805  11.76331226   0.60678736 -40.45208293
  31.52942085 -22.56989237  21.03679444 -12.03067669]
Accuracy is 0.8899082568807339


fold 2 of model 61 finished [  2.50274248

fold 0 of model 69 finished [-23.7309103   88.64102319  16.86848128   9.84388373 -67.14424763
  49.91634634 -39.10949182  34.74576918 -10.9495503 ]
Accuracy is 0.8348623853211009


fold 1 of model 69 finished [-48.56780672  77.97225969  22.76889347   1.89366164 -76.83627668
  61.46075499 -43.22011097  39.88436376 -20.80742358]
Accuracy is 0.8899082568807339


fold 2 of model 69 finished [-31.83816442  83.91927563   0.67172361  -4.1505019  -58.56750666
  51.90702157 -43.23446507  29.53282271 -12.87600932]
Accuracy is 0.8545454545454545


fold 3 of model 69 finished [-39.73535934  82.12484674   4.92835868   6.57033519 -68.55933112
  46.57436342 -38.91124591  30.08238055 -21.66112995]
Accuracy is 0.8623853211009175


fold 4 of model 69 finished [-31.01728232  83.57471225  23.75286855  11.33270382 -80.85199412
  54.14681735 -27.9895466   28.45050238 -25.95489635]
Accuracy is 0.8727272727272727
 Average accuracy is  0.862885738115096


[0.06, 1500]
fold 0 of model 70 finished [-27.06305492 

fold 3 of model 77 finished [-35.823385   101.74265296  -1.68259647   9.30407442 -80.54796946
  48.44228333 -45.52407577  35.80683023 -22.25386378]
Accuracy is 0.8807339449541285


fold 4 of model 77 finished [-49.18538426 100.21397971  31.35703323  14.99710512 -93.9105194
  67.32040888 -35.45344675  37.2390888  -37.06360236]
Accuracy is 0.8818181818181818
 Average accuracy is  0.8757130942452044


[0.09, 1000]
fold 0 of model 78 finished [-26.67716176 126.70642176  22.3787805   12.38246192 -99.60951309
  71.65556182 -58.15285275  48.32359015 -16.32125193]
Accuracy is 0.8348623853211009


fold 1 of model 78 finished [ -54.77011314   93.20514607   13.34561638   -4.07071145 -105.14652251
   64.36142321  -60.70532954   41.18256338  -26.52722831]
Accuracy is 0.8348623853211009


fold 2 of model 78 finished [ -67.95194228  142.57840292   16.38139892    0.64113905 -109.09801205
  107.85092458  -72.54666375   59.8179531   -32.36225406]
Accuracy is 0.8545454545454545


fold 3 of model 78 finis

array([ -5.90558826,  16.04379831,   3.03552162,   1.88314086,
       -12.06877259,  10.38581475,  -6.84997222,   6.83128821,
        -3.67196604])

In [12]:
    wfinal=findw(0.01,2000,X_train[0],y_train[0])
    a = predict(X_valid[0],wfinal)
    accu=accuracy(a,y_valid[0])
    co = conmat(a,y_valid[0])
    print(accu)
    print(co)

(109,)
(109,)
0.7706422018348624
[[34 23]
 [ 2 50]]
